In [1]:
import pandas as pd
import psycopg2

In [2]:
connection = psycopg2.connect(              
    database="startml",
    user="robot-startml-ro",
    password="pheiph0hahj1Vaif",
    host="postgres.lab.karpov.courses",
    port=6432
)
cursor = connection.cursor()      


Отберите уникальные значения тем (topic) в таблице постов (post).


Сделайте запрос на все колонки из таблицы user
SELECT * FROM "user"

In [3]:
cursor.execute("""                   
SELECT DISTINCT(topic) FROM "post"
""")

Отберите пользователей старше 30 с устройством на iOS. Каков ID и город записи с минимальным ID (он идет первым в таблице)? Запишите значения через пробел.

In [4]:
cursor.execute("""                   
SELECT age > 30, os = 'iOS'  FROM "user"
""")
results = cursor.fetchall()       # Получаем результаты (fetchall() - "получить всё")
results

[(True, False),
 (True, False),
 (False, False),
 (False, True),
 (True, False),
 (True, False),
 (False, False),
 (False, True),
 (False, False),
 (False, False),
 (True, False),
 (False, True),
 (True, True),
 (False, True),
 (False, False),
 (False, True),
 (True, False),
 (False, True),
 (False, True),
 (True, False),
 (True, False),
 (False, False),
 (True, True),
 (False, False),
 (True, True),
 (True, False),
 (True, True),
 (False, False),
 (False, True),
 (False, True),
 (True, False),
 (False, False),
 (False, True),
 (True, False),
 (False, False),
 (False, False),
 (True, False),
 (False, False),
 (False, False),
 (False, True),
 (False, False),
 (False, False),
 (True, False),
 (True, True),
 (True, False),
 (False, True),
 (False, False),
 (True, True),
 (False, False),
 (True, False),
 (False, True),
 (False, True),
 (False, True),
 (False, False),
 (True, False),
 (True, True),
 (True, True),
 (False, False),
 (True, True),
 (False, False),
 (False, False),
 (False, Tru

In [5]:
df = pd.read_sql(
    """
SELECT *
FROM "user"
WHERE age > 30 AND os = 'iOS';
 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,id,gender,age,country,city,exp_group,os,source
0,212,1,47,Russia,Podolsk,0,iOS,ads
1,222,0,45,Russia,Ulyanovsk,0,iOS,ads
2,224,1,32,Russia,Yekaterinburg,2,iOS,ads
3,226,1,31,Russia,Moscow,4,iOS,ads
4,243,1,38,Kazakhstan,Ekibastuz,1,iOS,ads
...,...,...,...,...,...,...,...,...
17665,168517,1,49,Russia,Urzhum,4,iOS,organic
17666,168525,1,39,Ukraine,Pavlohrad,1,iOS,organic
17667,168536,0,31,Russia,Michurinsk,1,iOS,organic
17668,168540,0,38,Russia,Tukan,0,iOS,organic


Отберите пользователей, которые не из России и при этом либо их экспериментальная группа не 0 и не 3 или их город – Минск.

In [6]:
df = pd.read_sql(
    """
SELECT *
FROM "user"
WHERE country != 'Russia' AND (exp_group NOT IN (0, 3) OR city = 'Minsk');


 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,id,gender,age,country,city,exp_group,os,source
0,211,1,22,Belarus,Minsk,0,iOS,ads
1,213,1,22,Ukraine,Kamianske,2,iOS,ads
2,216,0,38,Ukraine,Lviv,1,Android,ads
3,218,1,25,Belarus,Minsk,2,iOS,ads
4,228,0,17,Azerbaijan,Baku,2,iOS,ads
...,...,...,...,...,...,...,...,...
12471,168477,1,20,Turkey,Ankara,4,Android,organic
12472,168524,1,17,Turkey,Ereğli,2,Android,organic
12473,168525,1,39,Ukraine,Pavlohrad,1,iOS,organic
12474,168527,1,20,Ukraine,Sumy,2,iOS,organic


Подсчитайте средний возраст пользователей в разрезе страны. В ответ напишите средний возраст пользователей из Кипра. Ответ округлите до двух знаков после точки

In [7]:
df = pd.read_sql(
    """
SELECT ROUND(AVG(age),2), country
FROM "user"
GROUP BY country;


 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df


,round,country
0,26.78,Azerbaijan
1,27.28,Belarus
2,28.07,Cyprus
3,28.15,Estonia
4,27.13,Finland
5,27.22,Kazakhstan
6,27.22,Latvia
7,27.19,Russia
8,27.95,Switzerland
9,27.45,Turkey


Сгруппируйте пользователей сначала по экспериментальной группе, а затем по операционной системе. В каждой группе подсчитайте количество пользователей, максимальный и минимальный возраст.

Сохраните результаты в файл CSV с колонками в следующем порядке: exp_group, os, total_users, max_age, min_age (обратите внимание на название последних трех колонок!), используя запятую в качестве сепаратора, и отправьте файл в LMS.

In [8]:
df = pd.read_sql(
    """
SELECT exp_group, os, COUNT(*) AS total_users, MAX(age) AS max_age, MIN(age) AS min_age
FROM "user"
GROUP BY exp_group, os;


 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df_reset = df.reset_index()

df.to_csv('save.csv', index=False, sep= ',')

In [9]:
df

,exp_group,os,total_users,max_age,min_age
0,0,Android,21234,95,14
1,0,iOS,11489,79,14
2,1,Android,21232,92,14
3,1,iOS,11406,87,14
4,2,Android,21102,78,14
5,2,iOS,11512,74,14
6,3,Android,21319,84,14
7,3,iOS,11449,85,14
8,4,Android,21085,84,14
9,4,iOS,11377,78,14


В каждой категории постов определите длину самого длинного текста, используя таблицу post. В ответ отправьте тему, у которой длина самого большого поста больше 25 000.

In [10]:
df = pd.read_sql(
    """
SELECT topic, MAX(LENGTH(text)) AS max_text_length
FROM "post"
GROUP BY topic
HAVING MAX(LENGTH(text)) > 25000;




 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,topic,max_text_length
0,politics,25392


Найдите страну с наименьшим числом пользователей среди стран, у которых общее число пользователей больше 1000. Введите ее название на английском.
SELECT country, MIN(*) 
FROM "user"
GROUP BY country
HAVING COUNT(*) > 1000;

In [11]:
df = pd.read_sql(
    """
SELECT country
FROM "user"
GROUP BY country
HAVING COUNT(*) > 1000
ORDER BY COUNT(*) ASC
LIMIT 1;
 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,country
0,Azerbaijan


Количество пользователей внутри каждой группы, доступной в стране, для каждой страны

In [12]:
df = pd.read_sql(
    """
SELECT country, exp_group, COUNT(id)
FROM "user"
GROUP BY country, exp_group;
 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,country,exp_group,count
0,Azerbaijan,0,341
1,Azerbaijan,1,271
2,Azerbaijan,2,327
3,Azerbaijan,3,308
4,Azerbaijan,4,295
5,Belarus,0,671
6,Belarus,1,658
7,Belarus,2,644
8,Belarus,3,641
9,Belarus,4,679


Отберите для пользователей из Москвы экспериментальные группы, в которых средний возраст больше 27.2. Напишите через пробел число пользователей в этих группах.

In [13]:
df = pd.read_sql(
    """
SELECT COUNT(*)
FROM "user"
WHERE city = 'Moscow'
GROUP BY exp_group
HAVING AVG(age) > 27.2;
 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,count
0,4414
1,4378


Сейчас мы будем работать с другой таблицей под названием post. Найдите в таблице post 3 темы с наибольшим количеством постов. Напишите через пробел названия этих тем.

In [14]:
df = pd.read_sql(
    """
SELECT topic
FROM "post"
GROUP BY topic
ORDER BY COUNT(*) DESC
LIMIT 3;
 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,topic
0,movie
1,covid
2,sport


Отберите пользователей из Воронежа (Voronezh) и отсортируйте их по возрасту в убывающем порядке. Для равных возрастов отсортируйте по возрастанию экспериментальной группы.

Напишите ID второго в порядке выдачи пользователя.

In [15]:
df = pd.read_sql(
    """
SELECT *
FROM "user"
WHERE city = 'Voronezh'
ORDER BY age DESC, exp_group ASC;
 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,id,gender,age,country,city,exp_group,os,source
0,89990,0,71,Russia,Voronezh,0,Android,ads
1,2441,1,71,Russia,Voronezh,3,Android,ads
2,14184,1,66,Russia,Voronezh,3,Android,ads
3,35093,0,65,Russia,Voronezh,0,Android,ads
4,77284,0,64,Russia,Voronezh,4,Android,ads
...,...,...,...,...,...,...,...,...
1369,151841,1,14,Russia,Voronezh,4,Android,organic
1370,116446,0,14,Russia,Voronezh,4,iOS,organic
1371,18278,0,14,Russia,Voronezh,4,Android,ads
1372,110611,1,14,Russia,Voronezh,4,Android,organic


Отберите post_id, time, age и os пользователя для всех лайков пользователей из Омска. Отсортируйте результаты по убыванию времени действия и загрузите в LMS первые 100 значений в формате CSV (сепаратор "запятая", колонки с теми же названиями, что и в задании).

In [21]:
df = pd.read_sql(
    """
SELECT f.post_id, f.time, u.age, u.os
FROM "user" u
JOIN "feed_action" f ON f.user_id = u.id
WHERE u.city = 'Omsk' AND f.action = 'like'
ORDER BY f.time DESC
LIMIT 100;
 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df.to_csv('omsk1.csv', index=False, sep = ',')

Найдите город, пользователи которого в возрасте 36 лет 2021-12-01 числа просмотрели запись на тему covid меньше всего раз относительно других городов. Введите его название на английском.

In [35]:
df = pd.read_sql(
    """
SELECT u.city, COUNT(*) AS view_count
FROM "user" u
JOIN "feed_action" f ON f.user_id = u.id
JOIN "post" p ON p.id = u.id
WHERE u.age = 36 AND f.time::date = '2021-12-01' AND p.topic = 'covid'
GROUP BY u.city
ORDER BY view_count ASC
LIMIT 1;
 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,city,view_count
0,Dubove,16


In [39]:
df = pd.read_sql(
    """
SELECT u.city, COUNT(f.action) 
FROM "user" u
JOIN "feed_action" f ON f.user_id = u.id
JOIN "post" p ON f.post_id = p.id
WHERE u.age = 36 AND f.time::date = '2021-12-01' AND p.topic = 'covid' AND f.action = 'view'
GROUP BY u.city
ORDER BY count ASC
 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,city,count
0,Arkhangelsk,5
1,Apsheronsk,8
2,Fryazino,8
3,Varna,9
4,Karmaskaly,9
...,...,...
163,Nizhniy Novgorod,126
164,Belgorod,150
165,Novosibirsk,171
166,Saint Petersburg,230


Напишите SQL запрос, используя который вы получите:
- топ N постов по лайкам,
- количество пролайкавших пользователей,
- время последнего лайка.

В качестве ответа напишите через пробел первые 3 ID постов из этого топа.

In [48]:
df = pd.read_sql(
    """
SELECT post_id, MAX(f.action) as max_action, COUNT(f.user_id) as count_num, MAX(f.time) as lat_time
FROM "feed_action" f
JOIN "user" u ON f.user_id = u.id
JOIN "post" p ON f.post_id = p.id
WHERE f.action = 'like'
GROUP BY post_id
ORDER BY count_num DESC
LIMIT 10;
 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,post_id,max_action,count_num,lat_time
0,4866,like,31714,2022-01-01 13:57:43
1,5127,like,29637,2022-01-05 13:52:17
2,6448,like,27939,2022-01-22 05:50:39
3,6498,like,27729,2022-01-22 21:45:11
4,6513,like,27686,2022-01-22 19:59:52
5,3192,like,26507,2021-12-06 19:57:50
6,6314,like,26478,2022-01-20 07:46:21
7,6864,like,24156,2022-01-26 23:52:48
8,5413,like,23441,2022-01-07 09:57:07
9,5165,like,22596,2022-01-04 17:43:12


In [50]:
df = pd.read_sql(
    """
SELECT f.post_id, MAX(f.action) as max_action, COUNT(DISTINCT f.user_id) as count_num, MAX(f.time) as last_time
FROM "feed_action" f
JOIN "user" u ON f.user_id = u.id
JOIN "post" p ON f.post_id = p.id
WHERE f.action = 'like'
GROUP BY f.post_id
ORDER BY count_num DESC
LIMIT 10;

 """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

df

,post_id,max_action,count_num,last_time
0,4866,like,31714,2022-01-01 13:57:43
1,5127,like,29637,2022-01-05 13:52:17
2,6448,like,27939,2022-01-22 05:50:39
3,6498,like,27729,2022-01-22 21:45:11
4,6513,like,27686,2022-01-22 19:59:52
5,6314,like,26478,2022-01-20 07:46:21
6,3192,like,26251,2021-12-06 19:57:50
7,6864,like,24156,2022-01-26 23:52:48
8,5413,like,23441,2022-01-07 09:57:07
9,5165,like,22596,2022-01-04 17:43:12
